In [1]:
import numpy as np
import nltk
import torch
import sklearn
import re
import pyparsing
import copy


In [2]:
# Reading the provided file while removing the functional labels

file_name = "sequoia-corpus+fct.mrg_strict"

data = []
with open("./" + file_name, "r") as f:
    for sentence in f:
        sentence = sentence.strip()
        #sentence = filter_labels(sentence) ,can be done later and more easily   
        data.append(sentence)

In [3]:
print(data[0])

( (SENT (NP (NPP Gutenberg))))


In [4]:
def parse(expr):
    def _helper(s):
        items = []
        word = []
        for item in s:
            
            if word and (item == ' ' or item == ')' or item == '('):
                word = ''.join(word)
                items.append(word)
                word = []
                
            if item == '(':
                result, closeparen = _helper(s)
                if not closeparen:
                    raise ValueError("bad expression -- unbalanced parentheses")
                items.append(result)
            
            elif item == ')':
                return items, True
            
            elif item != ' ':
                word.append(item)
            
            else:
                pass
        
        return items, False
    return _helper(iter(expr))[0][0][0]

In [5]:
def get_tags(s, d):
    """Adds the current node's parent/child relation count to the probability dictionary"""
    tag_name = s[0].split("-")[0] # Remove any hiphen on tags
    if len(s) == 2 and isinstance(s[1], str):
        return
    childs = s[1:]
    child_tags = None
    
    for c in childs:
        child_tag_name = c[0].split("-")[0]
        if child_tags is None:
            child_tags =  child_tag_name
        else:
            child_tags = ','.join([child_tags, child_tag_name])
        get_tags(c, d)
    d[tag_name] = d.get(tag_name, {})
    d[tag_name][child_tags] = d[tag_name].get(child_tags, 0) + 1
    

In [6]:
def build_pcfg(sentences):
    """Builds the PCFG using the tagged sentences"""
    
    prob_dict = dict()
    tag_set = set()
    
    for s in sentences:
        parentheses_stack = []
        tags_stack = []
        s = parse(s)
        get_tags(s, prob_dict)
    
    for k1, d in prob_dict.items():
        tag_set.add(k1)
        tot_count = max(sum(d.values()), 1)
        for k2, v in d.items():
            tag_set.add(k2)
            d[k2] = v/tot_count
    
    return prob_dict, tag_set

In [7]:
pcfg, tag_set = build_pcfg(data)
print(pcfg)

{'NP': {'NPP': 0.061752433936022255, 'DET,NC': 0.1521557719054242, 'DET,ADJ,NC': 0.013574408901251738, 'ADJ,NC': 0.006787204450625869, 'DET,NC,AP': 0.040723226703755215, 'ADJ,NC,Ssub': 5.5632823365785815e-05, 'NPP,NPP': 0.021307371349095966, 'NC': 0.14403337969401947, 'DET,ADJ,NC,PP': 0.005285118219749653, 'DET,NPP': 0.016689847009735744, 'DET,NC,ADV,PP': 0.0003894297635605007, 'NC,PP': 0.054687065368567454, 'DET,ADJ,NC,COORD': 0.0003894297635605007, 'NC,NC,PP': 0.00033379694019471486, 'NPP,AP': 0.001780250347705146, 'NC,AP,PP': 0.007510431154381085, 'NPP,NPP,PONCT,NP,PONCT,VPpart': 0.00022253129346314326, 'PROREL': 0.02909596662030598, 'NC,COORD,PP,COORD': 5.5632823365785815e-05, 'NPP,PONCT,Srel,PONCT': 0.00011126564673157163, 'DET,NC,Srel': 0.0082336578581363, 'DET,NC,PP': 0.10019471488178025, 'DET,NC,NC': 0.002670375521557719, 'DET,NC,PP,PP': 0.010403337969401948, 'DET,NPP,PP': 0.0022809457579972183, 'DET,ADV,ADJ,VPinf': 0.00011126564673157163, 'DET,NC,NP': 0.0013908205841446453, 'N

In [8]:
def get_counts(s, d):
    """Adds the word count to the lexicon dictionary"""
    tag_name = s[0] # Do not remove hyphens
    if len(s) == 2 and isinstance(s[1], str):
        word = s[1].lower()
        d[word] = d.get(word, {})
        d[word][tag_name] = d[word].get(tag_name, 0) + 1
    childs = s[1:]
    for c in childs:
        get_counts(c, d)

In [9]:
def build_prob_lexicon(sentences):
    """Builds a probabilistic lexicon using the tagged sentences"""
    
    lexicon_dict = dict()
    
    for s in sentences:
        parentheses_stack = []
        tags_stack = []
        s = parse(s)
        get_counts(s, lexicon_dict)
    
    for k1, d in lexicon_dict.items():
        tot_count = max(sum(d.values()), 1)
        for k2, v in d.items():
            d[k2] = v/tot_count
    
    return lexicon_dict

In [10]:
lexicon = build_prob_lexicon(data)
print(lexicon)

{'gutenberg': {'NPP': 1.0}, 'cette': {'DET': 1.0}, 'exposition': {'NC': 1.0}, 'nous': {'CLO-A_OBJ': 0.07734806629834254, 'CLS-SUJ': 0.856353591160221, 'CLR': 0.04419889502762431, 'PRO': 0.011049723756906077, 'CLO-OBJ': 0.0055248618784530384, 'CLR-A_OBJ': 0.0055248618784530384}, 'apprend': {'V': 1.0}, 'que': {'CS': 0.7675, 'PROREL': 0.1725, 'PROREL-OBJ': 0.0025, 'ADV': 0.045, 'PROWH': 0.0125}, 'dès': {'P': 1.0}, 'le': {'DET': 0.9710610932475884, 'CLO-OBJ': 0.02652733118971061, 'CLO-ATS': 0.001607717041800643, 'NPP': 0.0008038585209003215}, 'e': {'l': 0.197078870496592, 'd': 0.602921129503408, 'L': 0.045180136319376826, 'c': 0.03291139240506329, 's': 0.033106134371957155, 'n': 0.034079844206426485, 'C': 0.005258033106134372, '3': 0.0023369036027263874, 'D': 0.0062317429406037, 'j': 0.016358325219084712, 'm': 0.005647517039922103, 'J': 0.016163583252190847, '8': 0.00019474196689386563, 'U': 0.0017526777020447906, 'N': 0.00019474196689386563, 'M': 0.00038948393378773126, '2': 0.00019474196

In [100]:
def remove_unit(cfg, A, d, k, tag):
    """Replaces the Unit rules by Multiple Rules by chaining"""
    B = tag
    new_entry = cfg.get(B)
    if new_entry is None:
        return False
    cfg[A].update(copy.deepcopy(new_entry))
    del cfg[B]
    del d[k]
    return True

In [142]:
def remove_multi(cfg, A, k, idx):
    """Replaces the Multiple Rules with chains of Double or Unit Rules"""
    tags = k.split(',')
    n = len(tags)
    d = cfg[A]
    
    new_tag = A + str(idx) + str(0)
    new_key = ','.join([tags[0], new_tag])
    d[new_key] = copy.deepcopy(d[k])
    del d[k]
    
    for i in range(1, n): 
        current_key = new_tag
        new_tag = A + str(idx) + str(i)
        cfg[current_key] =  {','.join([tags[i], new_tag]): 1.0}

In [143]:
def chomskyfy(context_free_grammar):
    """Binarisation of a CFG. Starts with removing Unit and then Multi Rules"""
    cfg = copy.deepcopy(context_free_grammar)
    
    unit_exist = True
    while unit_exist:
        unit_exist = False
        copy_cfg = copy.deepcopy(cfg) # Avoid Dictionnary size change during iteration
        for k1, d1 in copy_cfg.items():
            copy_d1 = copy.deepcopy(d1) # Avoid Dictionnary size change during iteration
            for k2, prob in copy_d1.items():
                tags = k2.split(',')[1:]
                if len(tags) == 1:
                    # Check that we have unit rules that we can remove
                    unit_exist = unit_exist or remove_unit(cfg, k1, d1, k2, tags[0])

    multi_exist = True
    while multi_exist:
        multi_exist = False
        copy_cfg = copy.deepcopy(cfg) # Avoid Dictionnary size change during iteration
        for k1, d1 in copy_cfg.items():
            copy_d1 = copy.deepcopy(d1) # Avoid Dictionnary size change during iteration
            for i, (k2, prob) in enumerate(copy_d1.items()):
                tags = k2.split(',')[1:]
                if len(tags) > 2:
                    multi_exist = True 
                    remove_multi(cfg, k1, k2, i)
    return cfg

In [144]:
binary_pcfg = chomskyfy(pcfg)
print(binary_pcfg)

{'SENT': {'NP': 0.11519845111326234, 'VN,Ssub,PONCT': 0.012262020006453695, 'ADVWH,NP,PONCT': 0.00032268473701193933, 'NP,PONCT': 0.04033559212649242, 'PP,PONCT': 0.002258793159083575, 'NP,VN,NP': 0.001379310344827586, 'NP,PONCT,Sint': 0.0006453694740238787, 'VN,NP,PONCT': 0.010648596321393998, 'VN,VPinf,PONCT': 0.015166182639561149, 'AP,PONCT': 0.0012907389480477573, 'COORD,PONCT': 0.011293965795417877, 'NP,PP': 0.0006453694740238787, 'ADV,PONCT': 0.00032268473701193933, 'NP,PP,PONCT': 0.0016134236850596966, 'VN,PP,PONCT': 0.005808325266214908, 'VPinf,PONCT': 0.006131010003226848, 'COORD': 0.0012907389480477573, 'Ssub,PONCT': 0.002258793159083575, 'PP,PONCT,NP': 0.00032268473701193933, 'NP,VN,PONCT': 0.006776379477250726, 'NP,Ssub,PONCT': 0.00032268473701193933, 'NP,VPinf': 0.00032268473701193933, 'NP,VN,ADV': 0.00032268473701193933, 'NP,PONCT,PP': 0.00032268473701193933, 'PONCT': 0.00032268473701193933, 'NP,PONCT,NP': 0.003872216844143272, 'NP,COORD,PONCT': 0.00032268473701193933, 'V

In [145]:
### CODE FROM THE POLYGLOT TUTORIAL ###

from operator import itemgetter
import re
import pickle

words, embeddings = pickle.load(open('polyglot-fr.pkl', 'rb'), encoding='latin1')

# Special tokens
Token_ID = {"<UNK>": 0, "<S>": 1, "</S>":2, "<PAD>": 3}
ID_Token = {v:k for k,v in Token_ID.items()}

# Map words to indices and vice versa
lexicon_words = lexicon.keys()
intersection_words = list()
intersection_embeddings = list()

for i, w in enumerate(words):
    if w in lexicon:
        intersection_words.append(w)
        intersection_embeddings.append(embeddings[i])

word_id = {w:i for (i, w) in enumerate(intersection_words)}
id_word = dict(enumerate(intersection_words))

# Normalize digits by replacing them with #
DIGITS = re.compile("[0-9]", re.UNICODE)


def case_normalizer(word, dictionary):
    """ In case the word is not available in the vocabulary,
     we can try multiple case normalizing procedure.
     We consider the best substitute to be the one with the lowest index,
     which is equivalent to the most frequent alternative."""
    w = word
    lower = (dictionary.get(w.lower(), 1e12), w.lower())
    upper = (dictionary.get(w.upper(), 1e12), w.upper())
    title = (dictionary.get(w.title(), 1e12), w.title())
    results = [lower, upper, title]
    results.sort()
    index, w = results[0]
    if index != 1e12:
        return w
    return word


def normalize(word, word_id):
    """ Find the closest alternative in case the word is OOV."""
    if not word in word_id:
        word = DIGITS.sub("#", word)
    if not word in word_id:
        word = case_normalizer(word, word_id)

    if not word in word_id:
        return None
    return word


def l2_nearest(embeddings, words, word_index, k):
    """Sorts words according to their Euclidean distance.
       To use cosine distance, embeddings has to be normalized so that their l2 norm is 1."""

    e1 = embeddings[word_index]
    for w2 in words:
        e2 = embeddings[word_id[w2]]
        distances.append(e1.dot(e2) / (np.norm(e1) * np.norm(e2)))
    sorted_distances = sorted(enumerate(distances), key=itemgetter(1))
    return zip(*sorted_distances[:k])


def knn(word, words, embeddings, word_id, id_word, k=5):
    word = normalize(word, word_id)
    if not word:
        print("OOV word")
        return
    word_index = word_id[word]
    indices, distances = l2_nearest(embeddings, words, word_index, k)
    neighbors = [id_word[idx] for idx in indices]
    
    return neighbors, distances

In [146]:
def levenstein_dist(w1, w2):
    """Computes the levesntein distance between w1 and w2"""
    
    m, n  = len(w1),len(w2)
    
    result = np.zeros((m, n))
    
    for i in range(m):
        result[i, 0] = i
    for i in range(n):
        result[0, i] = i
        
    for i, c1 in enumerate(w1):
        for j, c2 in enumerate(w2):
            if c1 == c2:
                result[i, j] = min(result[i-1, j] + 1,
                                   result[i, j-1] + 1,
                                   result[i-1, j-1],
                                  )
            else:
                result[i, j] = min(result[i-1, j] + 1,
                                   result[i, j-1] + 1,
                                   result[i-1, j-1] + 1,
                                  )
    return result[m-1, n-1]

In [149]:
def handle_oov(word, k=5):
    
    lev_dist = []
    for w2 in intersection_words:
        lev_dist.append(levenstein_dist(word, w2))
    min_indices = np.argpartition(lev_dist, k)[:k]
    close_words = intersection_words[min_indices]
    
    neighbors, distances = knn(word, words, intersection_embeddings, word_id, id_word)
    
    
    return neighbors[np.argmin(distances)]

In [152]:
def compute_cyk(sentence, use_oov=False):
    """Computes the CYK matrix for a sentence"""
    
    n = len(sentence)
    tag_list = list(tag_set)
    tag_to_idx = {tag: idx for (idx, tag) in enumerate(tag_list)}
    
    cyk_matrix = np.zeros((n, n, len(tag_list)))
    prob_matrix = np.zeros((n, n, len(tag_list)))
    
    # Unit words handling
    for i, w in enumerate(sentence):
        token_to_tag = word

        if not(word in intersection_words):
            if use_oov: 
                print(word+" is an OOV")
            token_to_tag = handle_oov(word)
            if use_oov:
                if token_to_tag is None:
                    print("No closest token found\n")
                else:
                    print("Closest token found :", token_to_tag, "\n")
        
        max_tag, max_prob = -1, -1
        for tag, prob in self.lexicon_inverted[token_to_tag].items():
            idx = tag_to_idx.get(tag)
            if not idx is None:
                prob_matrix[0, i , idx] = prob        
        
    
    # Strings of length 2 and more 
    for l in range(1, n):
        for start in range(n - l):
            for parent_tag in binary_pcfg.keys():
                for row in range(start, start + l):
                    for k1 in binary_pcfg[parent_tag].keys():
                        left_child_tag = k1.split(',')[0]
                        left_idx = tag_to_idx[left_child_tag]
                        proba_left = prob_matrix[row - start ,start, left_idx]
                        
                        if proba_left > prob_matrix[l, start, left_idx]:
                            
                            for k2, prob in binary_pcfg[parent_tag].items():
                                if left_child_tag != k2.split(',')[0]:
                                    continue #Skip non-possible grammar
                                right_child_tag = k1.split(',')[1]
                                right_idx = tag_to_idx[right_child_tag]
                                prob_right = prob_matrix[l - row - start - 1, row + 1, right_idx]
            
        
    

In [153]:
def compute_CYK_tables(sentence, viz_oov=False):
        # compute CYK tables :
        # - looking for the probabilities of the most likely trees parsing the substrings of sentence, for increasing length of substring (from 1 to length of the sentence)
        # - storing each time the position of the cut and the rule (right hand term) enabling to reach the most likely parsing tree of a given root tag

        nb_words = len(sentence)

        max_proba_derivation = np.zeros((nb_words, nb_words, self.PCFG.nb_all_symbols))
        # max_proba_derivation[s,l,a] is the maximum probability of
        # a parsing where symbol a derives substring x_s...x_(s+l)

        split_reaching_max = np.zeros((nb_words, nb_words, self.PCFG.nb_all_symbols, 3))
        # split_reaching_max[s,l,a,0] stores index cut
        # split_reaching_max[s,l,a,1] stores symbol b
        # split_reaching_max[s,l,a,2] stores symbol c
        # such that

        # (i) b derives x_s...x_(s+cut), c derives x_(s+cut)...x_(s+l)
        # and a rewrites bc (a->bc in the grammar)

        # (ii) the splitting <cut,b,c> defined by (i) is the one enabling
        # to reach the maximum probability for a to derives  x_s...x_(s+l)
        # (ie enabling to reach max_proba_derivation[s,l,a])

        # probabilities of tags for unary strings (words)
        for i, word in enumerate(sentence):

            token_to_tag = word

            if not(word in self.OOV.words_lexicon):
                if viz_oov: 
                    print(word+" is an OOV")
                token_to_tag = handle_oov(word)
                if viz_oov:
                    if token_to_tag is None:
                        print("No closest token found\n")
                    else:
                        print("Closest token found :",token_to_tag, "\n")

            if token_to_tag is None:
                for (tag,counts) in self.PCFG.freq_terminal_tags.items():
                    if tag in self.symbol_to_id:  # avoid the case where tag appearing in lexicon but not in grammar rules
                        id_tag = self.symbol_to_id[tag]
                        max_proba_derivation[i, 0, id_tag] = counts
            else:
                for (tag, proba) in self.lexicon_inverted[token_to_tag].items():
                    if tag in self.symbol_to_id: #avoid the case where tag appearing in lexicon but not in grammar rules
                        id_tag = self.symbol_to_id[tag]
                        max_proba_derivation[i, 0, id_tag] = proba

        for l in range(1, nb_words):
            # we will consider symbols deriving strings of length l+1...

            for s in range(nb_words - l):
                # ... and starting at index s of the sentence

                for idx_root_tag in self.grammar_ids:
                    # ... root_tag is the symbol deriving the considered string (rule left-hand term)

                    for cut in range(0, l):
                        # ... such symbol can rewrite as two symbols AB
                        # with A deriving substring until index cut included, and B deriving substring from index cut+1

                        for idx_left_tag in self.grammar_ids[idx_root_tag]: #left symbol A

                            proba_left_derivation = max_proba_derivation[s, cut, idx_left_tag]

                            if proba_left_derivation>max_proba_derivation[s, l, idx_root_tag]:

                                for (idx_right_tag, proba_split) in self.grammar_ids[idx_root_tag][idx_left_tag].items(): #right symbol B

                                    proba_right_derivation = max_proba_derivation[s + cut + 1, l - cut - 1, idx_right_tag]

                                    proba_decomposition = proba_split * proba_left_derivation * proba_right_derivation

                                    if proba_decomposition > max_proba_derivation[s, l, idx_root_tag]:
                                        # therefore, we found a new decomposition <cut,split[0],split[1]>
                                        # reaching a highest probability for root_tag to derive substring x_s...x_(s+l)

                                        max_proba_derivation[s, l, idx_root_tag] = proba_decomposition
                                        split_reaching_max[s, l, idx_root_tag, 0] = cut
                                        split_reaching_max[s, l, idx_root_tag, 1] = idx_left_tag
                                        split_reaching_max[s, l, idx_root_tag, 2] = idx_right_tag

        self.max_proba_derivation = max_proba_derivation
        self.split_reaching_max = split_reaching_max.astype(int)